In [1]:
pip install pflacco

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install cocopp

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cocoex
from pflacco.sampling import create_initial_sample
from pflacco.classical_ela_features import *
import pandas as pd

In [4]:
BBOB2 = cocoex.Suite("bbob", f"instances:1-5", f"function_indices:1-24 dimensions:2,3,5,10")
print(BBOB2)

Suite("b'bbob'", "b'instances:1-5'", "b'function_indices:1-24 dimensions:5,10'") with 240 problems in dimensions 5=10


In [6]:

features = []

for problem in BBOB2:
    dim = problem.dimension
    fid = problem.id_function
    iid = problem.id_instance

    print(f"fid:{fid}, dim:{dim}, iid:{iid}")
    
    # Create sample
    X = create_initial_sample(dim, sample_type = 'lhs', lower_bound = -5, upper_bound = 5)
    y = X.apply(lambda x: problem(x), axis = 1)
   
    

    # Classical ELA features
    ela_meta = calculate_ela_meta(X, y)
    ela_distr = calculate_ela_distribution(X, y)
    ela_level = calculate_ela_level(X, y, ela_level_resample_iterations = 3)

    # Compute the remaining 3 feature sets from the classical ELA features which do require additional function evaluations
    ela_local = calculate_ela_local(X, y, f = problem, dim = dim, lower_bound = -5, upper_bound = 5)
    ela_curv = calculate_ela_curvate(X, y, f = problem, dim = dim, lower_bound = -5, upper_bound = 5)
    ela_conv = calculate_ela_conv(X, y, f = problem)
    
    #Cell Mapping
    # Compute cell mapping angle feature set from the convential ELA features
    if(dim < 5):
        cm_angle = calculate_cm_angle(X, y, blocks = 3, lower_bound = -5, upper_bound = 5)
   
    
    
    #Dispersion features
    # Compute disp feature set from the convential ELA features
    disp = calculate_dispersion(X, y)
    
    
    #IC features
    # Compute ic feature set from the convential ELA features
    ic = calculate_information_content(X, y)
    
    #NBC features
    nbc = calculate_nbc(X, y)
    
    #Miscellaneous features
    #PCA features
    pca = calculate_pca(X,y)
    
    
    
    # cm_conv = calculate_cm_conv(X,y, lower_bound = -5, upper_bound = 5)
    # cm_grad = calculate_cm_grad(X,y, lower_bound = -5, upper_bound = 5)
    # limo = calculate_limo(X,y,lower_bound = -5, upper_bound = 5)
 
    data = pd.DataFrame({**{'fid': fid}, **{'dim': dim}, **{'iid': iid}, ** ela_meta,  **ela_distr, **ela_level, **ela_local, **ela_curv, **ela_conv, **disp, **ic, **nbc, **pca, **cm_angle}, index = [0])
    if(fid ==1 and iid == 1 and dim ==2):
        data.to_csv("data.csv",encoding='utf-8', index=False)
        print(data.shape)
    features.append(data)

#remove feature for dimensions with divide by zero error

ela_df = pd.concat(features).reset_index(drop = True)
print(f"Final row count of DataFrame: {ela_df.shape[0]}")
file_name = "features.csv"
ela_df.to_csv(file_name, encoding='utf-8', index=False)

fid:1, dim:5, iid:1


ValueError: The provided value for "block" is too large, resulting in less than 3 observations per cell.

In [ ]:
print("Done")

In [15]:
# Replace 'your_features.csv' with the actual path to your CSV file
file_path = 'features.csv'

# Load CSV into pandas DataFrame
df = pd.read_csv(file_path)


# Extract columns for which you want to calculate the median
columns_to_calculate_median = df.columns.difference(['fid', 'dim'])

# Define a custom median function to handle infinite values and empty slices
def custom_median(series):
    # Exclude NaN and infinite values
    valid_values = series.replace([np.inf, -np.inf], np.nan).dropna()
    
    # Check if there are valid values to calculate the median
    if len(valid_values) == 0:
        return np.nan
    
    return np.median(valid_values)


# Specify the cm_angle columns for which you want to calculate the mean
cm_angle_columns = [
    'cm_angle.angle_mean',
    'cm_angle.angle_sd',
    'cm_angle.costs_runtime',
    'cm_angle.dist_ctr2best_mean',
    'cm_angle.dist_ctr2best_sd',
    'cm_angle.dist_ctr2worst_mean',
    'cm_angle.dist_ctr2worst_sd',
    'cm_angle.y_ratio_best2worst_mean',
    'cm_angle.y_ratio_best2worst_sd'
]

# Group by fid, calculate the mean for selected cm_angle columns with dims 2 and 3
mean_values = result[result['dim'].isin([2, 3])].groupby(['fid'])[cm_angle_columns].mean()

# Create a new DataFrame with the mean values
mean_df = pd.DataFrame(mean_values).reset_index()
mean_df.columns = ['fid'] + [f'{col}_mean' for col in cm_angle_columns]

# Merge the mean_df with the original result DataFrame on fid
result = pd.merge(result, mean_df, on='fid', how='left')

# Update the values for dim 5 and 10 with the calculated means
for col in cm_angle_columns:
    result.loc[result['dim'].isin([5, 10]), col] = result[f'{col}_mean']

# Drop the unnecessary columns
result = result.drop([f'{col}_mean' for col in cm_angle_columns], axis=1)




file_name = 'median_features.csv'
result.to_csv(file_name, encoding='utf-8', index=False)



